In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
import glob

import math
from scipy import linalg
from sklearn import linear_model, datasets

In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

In [3]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        
        # termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,6), corners,ret)
        cv2.imshow('img',img)
        cv2.waitKey(5000)
    
cv2.destroyAllWindows()

In [4]:
print(corners[:1])

[[[146.39856 240.75754]]]


Progress: We have the image and object co-ordinates

Next we are fitting Ransac without removing z points

In [5]:
imgp = np.ones((3,42))

imgp1 = np.array(imgpoints)
imgp1 = imgp1.reshape((2,42))
imgp[:2,:] = imgp1
print(imgp.shape)

objp = objp.T
print(objp.shape)
obj = np.ones((4,42))
obj[:3,:] = objp

(3, 42)
(3, 42)


In [6]:
print(obj)
print(obj.shape)

[[0. 1. 2. 3. 4. 5. 6. 0. 1. 2. 3. 4. 5. 6. 0. 1. 2. 3. 4. 5. 6. 0. 1. 2.
  3. 4. 5. 6. 0. 1. 2. 3. 4. 5. 6. 0. 1. 2. 3. 4. 5. 6.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3.
  3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 5. 5. 5. 5. 5. 5. 5.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
(4, 42)


In [7]:
print(imgp1)

[[146.39856  240.75754  148.83553  209.49371  151.51917  180.04099
  154.09694  152.48091  156.50087  126.542336 158.9237   102.02474
  161.3021    78.796326 177.5049   240.6578   179.30621  208.8729
  180.8149   179.32913  182.54639  151.4929   184.2548   125.364655
  185.76144  100.6497   187.3481    77.39799  209.64189  240.22723
  210.36389  208.30289  210.88799  178.40152  211.65027  150.34837
  212.41748  124.126366 213.21379   99.3735   213.91742   76.18416 ]
 [242.29851  239.7209   241.74269  207.66035  241.53677  177.55673
  241.34033  149.31512  241.14352  123.01466  240.99368   98.10075
  240.7799    74.75045  275.28555  239.39507  273.88638  206.89265
  272.51752  176.77252  271.35648  148.47687  270.1829   121.888
  269.16333   96.91115  267.94495   73.441795 308.6997   238.7351
  306.31305  206.26714  303.97327  175.84785  301.582    147.48137
  299.47217  120.78554  297.42093   95.72883  295.4988    72.44874 ]]


In [8]:
imgp1
imgx = imgp1[0,:]
imgy = imgp1[1,:]
imgx

array([146.39856 , 240.75754 , 148.83553 , 209.49371 , 151.51917 ,
       180.04099 , 154.09694 , 152.48091 , 156.50087 , 126.542336,
       158.9237  , 102.02474 , 161.3021  ,  78.796326, 177.5049  ,
       240.6578  , 179.30621 , 208.8729  , 180.8149  , 179.32913 ,
       182.54639 , 151.4929  , 184.2548  , 125.364655, 185.76144 ,
       100.6497  , 187.3481  ,  77.39799 , 209.64189 , 240.22723 ,
       210.36389 , 208.30289 , 210.88799 , 178.40152 , 211.65027 ,
       150.34837 , 212.41748 , 124.126366, 213.21379 ,  99.3735  ,
       213.91742 ,  76.18416 ], dtype=float32)

In [9]:
# objx = objp[0,:]
# objy = objp[1,:]

In [10]:
# objx.reshape((-1,1))
# imgx.reshape((-1,1))
# objx.shape
# imgx.shape

In [9]:
X = obj.T
y = imgp.T

In [10]:
print(X.shape)
print(y.shape)

(42, 4)
(42, 3)


In [12]:
lr = linear_model.LinearRegression()
lr.fit(X, y)

ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

l_lr = lr.predict(X)
l_rs = ransac.predict(X)

# Compare estimated coefficients
print("Estimated coefficients (linear regression):")
print(lr.coef_)

print("Estimated coefficients (RANSAC):")
print(ransac.estimator_.coef_)

Estimated coefficients (linear regression):
[[-6.18933569  2.26181529  0.          0.        ]
 [-7.00757036  0.96638078  0.          0.        ]
 [ 0.          0.          0.          0.        ]]
Estimated coefficients (RANSAC):
[[-0.84222789 11.38212451  0.          0.        ]
 [-1.90793011 13.73601381  0.          0.        ]
 [ 0.          0.          0.          0.        ]]


Ransac after removing Z points

In [13]:
imgp = np.ones((3,42))

imgp1 = np.array(imgpoints)
imgp1 = imgp1.reshape((2,42))
imgp[:2,:] = imgp1
print(imgp.shape)

obj1 = np.ones((3,42))
obj1[:2,:] = objp[:2,:]

(3, 42)


In [14]:
print(obj1)
print(obj1.shape)

[[0. 1. 2. 3. 4. 5. 6. 0. 1. 2. 3. 4. 5. 6. 0. 1. 2. 3. 4. 5. 6. 0. 1. 2.
  3. 4. 5. 6. 0. 1. 2. 3. 4. 5. 6. 0. 1. 2. 3. 4. 5. 6.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3.
  3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 5. 5. 5. 5. 5. 5. 5.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
(3, 42)


In [19]:
print(imgp.shape)

(3, 42)


In [23]:
# imgp

In [20]:
X1 = obj1.T
y1 = imgp.T

In [21]:
print(X.shape)
print(y.shape)

(42, 3)
(42, 3)


In [22]:
lr = linear_model.LinearRegression()
lr.fit(X1, y1)

ransac = linear_model.RANSACRegressor()
ransac.fit(X1, y1)
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

l_lr = lr.predict(X)
l_rs = ransac.predict(X)

# Compare estimated coefficients
print("Estimated coefficients (linear regression):")
print(lr.coef_)

print("Estimated coefficients (RANSAC):")
print(ransac.estimator_.coef_)

Estimated coefficients (linear regression):
[[-6.18933569  2.26181529  0.        ]
 [-7.00757036  0.96638078  0.        ]
 [ 0.          0.          0.        ]]
Estimated coefficients (RANSAC):
[[-0.17461111 11.80779471  0.        ]
 [ 0.65572807 15.37059359  0.        ]
 [ 0.          0.          0.        ]]


NO progress!